In [1]:
!pip install git+https://github.com/pswpswpsw/nif.git

  Cloning https://github.com/pswpswpsw/nif.git to /tmp/pip-req-build-ot9vite_
  Running command git clone -q https://github.com/pswpswpsw/nif.git /tmp/pip-req-build-ot9vite_
  Created wheel for NIF: filename=NIF-1.0.0-py3-none-any.whl size=2943257 sha256=b7efe2cc8185a66b01ac6ee4fb39de88256cf0759eb6d93891382e46a1a9d87c
  Stored in directory: /tmp/pip-ephem-wheel-cache-q29dalrg/wheels/88/ba/9a/716e4d26db4b1364836a1dd0bfa96bb8d89b1e67736b0da3fb
Successfully built NIF


In [2]:
import tensorflow as tf
import nif

1 Physical GPUs, 1 Logical GPUs


# How to get the gradients: dy/dx?

- Note: 
    - since we wrap up everything in keras.Model, we want to keep things as simple as possible as well for getting the gradient

## Construct a keras.Model 

In [3]:
x = tf.keras.Input(4,)
l1 = tf.keras.layers.Dense(5, activation='tanh')
model = tf.keras.Model([x], [l1(x)])

## Import `JacobianLayer` and `HessianLayer`

In [4]:
from nif.layers import JacobianLayer, HessianLayer

## You can also decide what indices of both output $y$ and input $x$ that you want to get the gradients on

- here you define them as `y_index` and `x_index`

In [5]:
x_index = [0,1,2,3]
y_index = [0,1,2,3,4]

## Now we can use `nif.layers.JacobianLayer` to **wrap** up the `keras.Model` 

In [6]:
y_and_dydx_layer = JacobianLayer(model, y_index,x_index)

# Get the gradient tensor by passing input to the `JacobianLayer` instance
- you can see there are two outputs, first one is `y`, second one is `dy/dx`

In [7]:
x_data = tf.random.uniform((10,4))
y_pred, dydx = y_and_dydx_layer(x_data)

## We can also verify the shape, it is `(batch_size, dim(y_chosen), dim(x_chosen))`

In [8]:
print("f(x) shape = ", y_pred.shape)
print("df(x)/dx shape = ", dydx.shape)

f(x) shape =  (10, 5)
df(x)/dx shape =  (10, 5, 4)


# We can even compute Hessian, the second order partial derivatives, for any `keras.Model`

In [9]:
y_and_dydx_and_dy2dx2_layer = HessianLayer(model, y_index, x_index)
y_pred, dydx, dy2dx2 = y_and_dydx_and_dy2dx2_layer(x_data)

## we can also verify the shape, it is `(batch_size, dim(y_chosen), dim(x_chosen), dim(x_chosen))`

In [10]:
print("d2f(x)/dx2 shape = ", dy2dx2.shape)

d2f(x)/dx2 shape =  (10, 5, 4, 4)


# Why do we need this?

- we have input $x,t$ and output $u(x,t)$ for typical spatial-temporal dynamics
- when we are using PDE information, we need things like $\partial{u}/\partial{x} + \partial{u}/\partial{t}=0$
- they can be embedded elegantly inside the `keras.Model` **in the form of layer regularization loss**.